In [1]:
import pandas as pd
from datetime import datetime

In [2]:
data_path = "./m_a_tkg/update_task/"

### Initial TKG

In [3]:
entities_list = pd.read_csv(data_path + 'entities_list_wiki_with_wikidata_0109.csv')

In [4]:
import ast
entities_list['qa_data'] = entities_list['qa_data'].apply(lambda x: ast.literal_eval(x))
entities_list['eda_data'] = entities_list['eda_data'].apply(lambda x: ast.literal_eval(x))

In [5]:
wikidata_with_tuples = entities_list[entities_list['len_qa_data'] != 0]
wikidata_with_tuples['qa_data'] = wikidata_with_tuples['qa_data'].apply(lambda x: list(dict.fromkeys(x)))
wikidata_with_tuples['eda_data'] = wikidata_with_tuples['eda_data'].apply(lambda x: list(dict.fromkeys(x)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
ground_zero = list(zip(wikidata_with_tuples['lowered_entity_names'], wikidata_with_tuples['qa_data']))

In [7]:
tkg_dict = dict(ground_zero)
all_tuples = []
for key, item in tkg_dict.items():
    all_tuples.append(item)
flat_list = [item for sublist in all_tuples for item in sublist]

In [8]:
initial_tkg = pd.DataFrame(flat_list, columns=['head_entity', 'relation', 'tail_entity', 'start_date'])
initial_tkg['end_date'] = None

In [9]:
initial_tkg

,head_entity,relation,tail_entity,start_date,end_date
0,apple_inc.,owner of (P1830),Imagination Technologies (Q1119662),2018-01-01T00:00:00Z,None
1,apple_inc.,owner of (P1830),Apple corporate shuttle (Q110553619),2018-01-01T00:00:00Z,None
2,apple_inc.,subsidiary (P355),"FileMaker, Inc. (Q1982831)",2018-01-01T00:00:00Z,None
3,apple_inc.,subsidiary (P355),Anobit (Q2893391),2018-01-01T00:00:00Z,None
4,apple_inc.,subsidiary (P355),Beats Electronics (Q1961036),2018-01-01T00:00:00Z,None
...,...,...,...,...,...
8349,sm_entertainment,subsidiary (P355),SM Classics (Q105753127),2018-01-01T00:00:00Z,None
8350,sm_entertainment,subsidiary (P355),Label V (Q107463954),2018-01-01T00:00:00Z,None
8351,sm_entertainment,subsidiary (P355),Baljunso (Q18159635),2018-01-01T00:00:00Z,None
8352,sm_entertainment,subsidiary (P355),SM Life Design Group (Q107473707),2018-01-01T00:00:00Z,None


### News Articles (and check if sentences are in the correct relation and time order)

In [10]:
sentence_df = pd.read_csv(data_path + 'new_sentence_df_0509.csv')

In [11]:
# sentence_df['tuples'] = sentence_df['tuples'].apply(lambda x: ast.literal_eval(x))
# sentence_df['publishedAt'] = sentence_df['publishedAt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [12]:
# # Check if relation and time ordering is correct in sentence_df
# sentence_df.drop_duplicates('tuples', inplace = True)
# tuples_index_2018_df = sentence_df.set_index(['tuples'])
# sorted_df = tuples_index_2018_df.sort_values('publishedAt')
# sorted_df.index.nunique()

# relation_ordering = {
#     'considering': 1,
#     'expecting': 2,
#     'success': 3,
#     'terminated': 4
# }

# def check_relation_ordering(dataframe, relation_order = relation_ordering):
#     current = 0
#     for indx in dataframe.index:
#         relation = indx[1].split('_')[0]
#         if relation_order[relation] > current:
#             current = relation_order[relation]
#         else:
#             return False
#     return True

# def check_time_ordering(dataframe):
#     latest_date = None
#     for idx, item in enumerate(dataframe['publishedAt']):
#         if latest_date == None:
#             latest_date = item
#         elif item < latest_date:
#             return False
#         elif item >= latest_date:
#             latest_date = item
#     return True

# res = []
# for i in tuples_index_2018_df['final_company_deal'].unique():
#     temp_df = tuples_index_2018_df[tuples_index_2018_df['final_company_deal'] == i]
#     if not check_relation_ordering(temp_df):
#         res.append((i, False))
#     else:
#         res.append((i, True))
        
# time_res = []
# for i in tuples_index_2018_df['final_company_deal'].unique():
#     temp_df = tuples_index_2018_df[tuples_index_2018_df['final_company_deal'] == i]
#     if not check_time_ordering(temp_df):
#         time_res.append((i, False))
#     else:
#         time_res.append((i, True))
        
# relation_res_df = pd.DataFrame(res)
# time_res_df = pd.DataFrame(time_res)

# print(relation_res_df[1].value_counts())
# print(time_res_df[1].value_counts())

### Parse each document now from sorted_df

In [13]:
sentence_df

,tuples,article_id,sentences,relation,publishedAt,merger_or_acq,keywords,mentions,relations,wikidata,...,final_company_deal,general_questions,head_questions,tail_questions,new_general_questions,new_head_questions,new_tail_questions,latest_general_questions,latest_head_questions,latest_tail_questions
0,"('apple_inc.', 'considering_acq', 'netflix', '...",1.0,There is a 40% likelihood that Apple will acqu...,considering_acq,2018-01-01 09:04:47,Acq,acquire,"[('Citi', 'Org'), ('Apple', 'Bidder'), ('Netfl...",['40% likelihood'],True,...,apple_inc._netflix,"((""What's the deal between apple_inc. and netf...",NaN,NaN,"[(""What's the deal between apple_inc. and netf...",NaN,NaN,"[(""Who's the bidder of the acquisition deal on...",NaN,NaN
1,"('prestige_group', 'success_acq', 'capitaland'...",2.0,NEW DELHI: Realty firm Prestige group today sa...,success_acq,2018-01-01 12:46:26,Acq,acquire,"[('Prestige group', 'Bidder'), ('it will acqui...",['it will acquire'],False,...,prestige_group_capitaland,"((""What's the deal between prestige_group and ...","{'before': [], 'after': []}","{'before': [], 'after': []}","[(""What's the deal between prestige_group and ...","{'before': [], 'after': []}","{'before': [], 'after': []}","[(""Who's the bidder of the acquisition deal on...","{'before': [], 'after': []}","{'before': [], 'after': []}"
2,"('bd_(company)', 'success_acq', 'bard', '2018-...",5.0,"Under the terms of the transaction, upon compl...",success_acq,2018-01-02 00:00:00,Acq,acquisition,"[('Bard', 'Target'), ('Bard', 'Target'), ('BD'...",['upon completion of the acquisition'],False,...,bd_(company)_bard,"((""What's the deal between bd_(company) and ba...",{'before': [('Who does FlowJo LLC (Q106573956)...,"{'before': [], 'after': []}","[(""What's the deal between bd_(company) and ba...",{'before': [('Who does FlowJo LLC (Q106573956)...,"{'before': [], 'after': []}","[(""Who's the bidder of the acquisition deal on...",{'before': [('Who does FlowJo LLC (Q106573956)...,"{'before': [], 'after': []}"
3,"('bhi', 'success_merger', 'marcventure', '2018...",19.0,Marcventure said the merger with BHI will enab...,success_merger,2018-01-02 09:09:57,Merger,merger,"[('BHI', 'Bidder'), ('Marcventure', 'Bidder'),...",['merger'],False,...,bhi_marcventure,"((""What's the deal between bhi and marcventure...","{'before': [], 'after': []}","{'before': [], 'after': []}","[(""What's the deal between bhi and marcventure...","{'before': [], 'after': []}","{'before': [], 'after': []}","[(""Who's the first bidder of the merger deal o...","{'before': [], 'after': []}","{'before': [], 'after': []}"
4,"('h.j. heinz', 'success_merger', 'kraft_foods'...",22.0,"For two years, investors and other stakeholder...",success_merger,2018-01-02 10:15:00,Merger,merger,"[('Kraft Heinz', 'Org'), ('H.J. Heinz', 'Bidde...",['created through the $46 billion merger'],False,...,h.j. heinz_kraft_foods,"((""What's the deal between h.j. heinz and kraf...","{'before': [], 'after': []}",{'before': [('Who owns Kool-Aid (Q1531983) bef...,"[(""What's the deal between h.j. heinz and kraf...","{'before': [], 'after': []}",{'before': [('Who owns Kool-Aid (Q1531983) bef...,"[(""Who's the first bidder of the merger deal o...","{'before': [], 'after': []}",{'before': [('Who owns Kool-Aid (Q1531983) bef...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5716,"('tervita corporation', 'expecting_merger', 't...",53598.0,"(""SECURE"") (TSX: SES) and Tervita Corporation ...",expecting_merger,2021-06-30 06:17:00,Merger,merger,"[('Tervita Corporation', 'Bidder'), ('TSX', 'O...",['proposed merger'],False,...,tervita corporation_tervita,"((""What's the deal between tervita corporation...",NaN,NaN,"[(""What's the deal between tervita corporation...",NaN,NaN,"[(""Who's the first bidder of the merger deal o...",NaN,NaN
5717,"('american_lithium_energy_(united_states)', 's...",53613.0,(American Lithium or the Company) (TSX-V: LI |...,success_merger,2021-06-30 11:00:00,Merger,merge

In [14]:
def return_common_entities(tuple_list1, tuple_list2):
    entities_list1 = [i[2] for i in tuple_list1]
    entities_list2 = [i[2] for i in tuple_list2]
    
    list1_as_set = set(entities_list1)
    intersection = list1_as_set.intersection(entities_list2)

    intersection_as_list = list(intersection)

    return intersection_as_list

In [15]:
import copy

all_tkg_snapshots = [tkg_dict]

In [16]:
for i in range(len(sentence_df)):
    data = sentence_df.iloc[i]
    head_entity = data['lowered_head']
    relation_word, deal = data['relation'].split('_')[0], data['relation'].split('_')[1]
    tail_entity = data['lowered_tail']
    date = data['publishedAt'].split(" ")
    date = date[0] + 'T' + date[1] + 'Z'
    quad = (head_entity, data['relation'], tail_entity, date)
    
    latest_dict = copy.deepcopy(all_tkg_snapshots[-1])
    
    try:
        head_data = latest_dict[head_entity]
        tail_data = latest_dict[tail_entity]
    except:
        if head_entity not in latest_dict.keys():
            latest_dict[head_entity] = []

        if tail_entity not in latest_dict.keys():
            latest_dict[tail_entity] = []
        
        head_data = latest_dict[head_entity]
        tail_data = latest_dict[tail_entity]
    
    common_entities = return_common_entities(head_data, tail_data)
    
    new_head_tuples = []
    new_tail_tuples = []
    to_be_remove_tuples = []
    if relation_word == 'success':
        if deal == 'acq':
            # success_acq
            for d in tail_data:
                if d[1] == 'owner of (P1830)' or d[1] == 'subsidiary (P355)' or d[1] == 'business division (P199)':
                    # the head_entity now acquires certain assets of the tail and new quads are being added
                    new_tuple = (head_entity, d[1], d[2], date)
                    new_head_tuples.append(new_tuple)

                    # the tail entity is being acquired so all tuples now include an end date
                    end_tuple = d + (date,)
                    new_tail_tuples.append(end_tuple)
                    to_be_remove_tuples.append(d)
                elif d[1] == 'board member (P3320)' or d[1] == 'owned by (P127)':
                    if not d[2] in common_entities:
                        # the tail entity is being acquired so all tuples now include an end date
                        end_tuple = d + (date,)
                        new_tail_tuples.append(end_tuple)
                        to_be_remove_tuples.append(d)

            for h in head_data:
                # the tail entity is being acquired so all the board member and owned by values now own the tail entity
                if h[1] == 'board member (P3320)' or h[1] == 'owned by (P127)':
                    new_tuple = (tail_entity, h[1], h[2], date)
                    new_tail_tuples.append(new_tuple)
                    
                if h[2] == tail_entity and len(h) == 4: # length 4 means only has start date and no end date:
                    end_tuple = h + (date,)
                    new_head_tuples.append(end_tuple)
                    to_be_remove_tuples.append(h)
        elif deal == 'merger':
            # success_merger            
            for h in head_data:
                if h[2] == tail_entity and len(h) == 4: # length 4 means only has start date and no end date
                    end_tuple = h + (date,)
                    new_head_tuples.append(end_tuple)
                    to_be_remove_tuples.append(h)
    else:
        # all other relations
        for h in head_data:
            if h[2] == tail_entity and len(h) == 4: # length 4 means only has start date and no end date
                end_tuple = h + (date,)
                new_head_tuples.append(end_tuple)
                to_be_remove_tuples.append(h)
    
    # remove outdated tuples
    for z in to_be_remove_tuples:
        try:
            latest_dict[tail_entity].remove(z)
        except:
            latest_dict[head_entity].remove(z)

    # add new tuples
    latest_dict[head_entity].append(quad)
    latest_dict[head_entity] = latest_dict[head_entity] + new_head_tuples
    latest_dict[tail_entity] = latest_dict[tail_entity] + new_tail_tuples
    
    all_tkg_snapshots.append(latest_dict)
    

In [17]:
def cal_size_tkg_dict(snapshot_tkg):
    emp_length = 0
    for x in snapshot_tkg.values():
        if isinstance(x, list):
            emp_length += len(x)
    
    return emp_length

### Compute differences between snapshots

In [18]:
from deepdiff import DeepDiff

all_differences = []
for i in range(len(all_tkg_snapshots)):
    if i == 0:
        all_differences.append({})
        continue
    
    previous_tkg_snapshot = all_tkg_snapshots[i - 1]
    latest_tkg_snapshot = all_tkg_snapshots[i]
    
    all_differences.append(DeepDiff(previous_tkg_snapshot, latest_tkg_snapshot))

In [19]:
all_differences = all_differences[1:]

In [20]:
initial_tkg = all_tkg_snapshots[0]

In [21]:
len_tkg_snapshots = [cal_size_tkg_dict(i) for i in all_tkg_snapshots]

In [22]:
len(all_tkg_snapshots)

5722

In [23]:
len(len_tkg_snapshots)

5722

In [24]:
len(all_differences)

5721

In [25]:
len(sentence_df)

5721

### Flatten dictionary and convert to ids

In [26]:
def flatten_dict(dictionary):
    res = []
    for key, value in dictionary.items():
        res += value
    
    return res

In [27]:
def remove_none_tuples(list_of_tuples):
    for item in list_of_tuples:
        if type(item[2]) == type(None) or type(item[0]) == type(None) or type(item[1]) == type(None) or type(item[3]) == type(None):
            list_of_tuples.remove(item)

In [28]:
flatten_all_tkg_snapshots = [flatten_dict(x) for x in all_tkg_snapshots]

In [29]:
len(flatten_all_tkg_snapshots[0])

8354

In [30]:
cleaned_all_tkg_snapshots = [remove_none_tuples(x) for x in flatten_all_tkg_snapshots]

In [31]:
len(flatten_all_tkg_snapshots[0])

8319

In [32]:
sentence_df['before_tkg'] = flatten_all_tkg_snapshots[:5721]

In [33]:
sentence_df['after_tkg'] = flatten_all_tkg_snapshots[1:]

In [34]:
sentence_df['differences'] = all_differences

### Create ent_id, rel_id, and ts_id

In [166]:
def create_ent_rel_ts_id_at_t(tkg_snapshot):
    tuples_at_t = []
    for key, item in tkg_snapshot.items():
        tuples_at_t.append(item)
    flat_list = [item for sublist in tuples_at_t for item in sublist]

    try:
        t_state_tkg = pd.DataFrame(flat_list, columns=['head_entity', 'relation', 'tail_entity', 'start_date', 'end_date'])
    except:
        t_state_tkg = pd.DataFrame(flat_list, columns=['head_entity', 'relation', 'tail_entity', 'start_date'])
        t_state_tkg['end_date'] = None
    
    def creating_time_point(dataframe):
        res = []
        for i in range(len(dataframe)):
            if pd.isna(dataframe['end_date'][i]):
                res.append('2021-07-01T00:00:00+00:00Z')
            else:
                res.append(dataframe['end_date'][i])

        return res
    
    t_state_tkg['new_end_date'] = creating_time_point(t_state_tkg)
    t_state_tkg['time_point_de'] = t_state_tkg['new_end_date'].apply(lambda x: x.split('T')[0])
    
    entities, relations = set(), set()
    for i in range(len(t_state_tkg)):
        data = t_state_tkg.iloc[i]
        head_entity = data['head_entity']
        tail_entity = data['tail_entity']
        relation = data['relation']

        entities.add(head_entity)
        entities.add(tail_entity)
        relations.add(relation)
        
    entities.remove(None)
    entities.add('ANSWER HOLDER')
    
    a = list(t_state_tkg['start_date']) + list(t_state_tkg['new_end_date'])
    b = [x.split('T')[0] for x in a]
    c = list(set(b))
    c.sort()
    ts_res = []
    for x in c:
        date_split = x.split('-')
        ts_res.append((int(date_split[0]), int(date_split[1]), int(date_split[2])))
    
    entities_to_id = {x: i for (i, x) in enumerate(sorted(entities))}
    relations_to_id = {x: i for (i, x) in enumerate(sorted(relations))}
    timestamps_to_id = {x: i for (i, x) in enumerate(ts_res)}
    
    return entities_to_id, relations_to_id, timestamps_to_id, t_state_tkg

In [167]:
id_results = []
for snap in all_tkg_snapshots:
    res = {}
    ent, rel, ts, df = create_ent_rel_ts_id_at_t(snap)
    res['ent_id'] = ent
    res['rel_id'] = rel
    res['ts_id'] = ts
    id_results.append(res)

In [168]:
len(id_results)

5722

In [35]:
all_tuples = []
for key, item in all_tkg_snapshots[-1].items():
    all_tuples.append(item)
flat_list = [item for sublist in all_tuples for item in sublist]

last_state_tkg = pd.DataFrame(flat_list, columns=['head_entity', 'relation', 'tail_entity', 'start_date', 'end_date'])

In [36]:
entities, relations = set(), set()
for i in range(len(last_state_tkg)):
    data = last_state_tkg.iloc[i]
    head_entity = data['head_entity']
    tail_entity = data['tail_entity']
    relation = data['relation']
    
    entities.add(head_entity)
    entities.add(tail_entity)
    relations.add(relation)

In [37]:
len(entities)

14757

In [38]:
entities.remove(None)
entities.add('ANSWER HOLDER')

In [39]:
len(entities)

14757

In [40]:
len(relations)

13

In [41]:
times = pd.date_range('2018-01-01', periods=(1278.5), freq='1D')
string_dates = [x.strftime('%Y-%m-%d') for x in times]

ts_res = []
for x in string_dates:
    date_split = x.split('-')
    ts_res.append((int(date_split[0]), int(date_split[1]), int(date_split[2])))

In [42]:
entities_to_id = {x: i for (i, x) in enumerate(sorted(entities))}
relations_to_id = {x: i for (i, x) in enumerate(sorted(relations))}
timestamps_to_id = {x: i for (i, x) in enumerate(ts_res)}

In [43]:
import os
import pickle
for (dic, f) in zip([entities_to_id, relations_to_id, timestamps_to_id], ['ent_id_1010', 'rel_id_1010', 'ts_id_1010']):
    ff = open(os.path.join(data_path, f), 'wb')
    pickle.dump(dic, ff)
    ff.close()

### Prepare data for update process

In [44]:
import pickle
dicts = {}
for f in ['ent_id_1010', 'rel_id_1010', 'ts_id_1010']:
    in_file = open(str(data_path + f), 'rb')
    dicts[f] = pickle.load(in_file)

In [45]:
sentence_df['latest_general_questions'] = sentence_df['latest_general_questions'].apply(lambda x: ast.literal_eval(x) if type(x) == type("string") else x)
sentence_df['latest_head_questions'] = sentence_df['latest_head_questions'].apply(lambda x: ast.literal_eval(x) if type(x) == type("string") else x)
sentence_df['latest_tail_questions'] = sentence_df['latest_tail_questions'].apply(lambda x: ast.literal_eval(x) if type(x) == type("string") else x)

In [73]:
def encoding_tkg(list_tkg):
    quad_ids = []
    for x in list_tkg:
        head_id = dicts['ent_id_1010'][x[0]]
        relation_id = dicts['rel_id_1010'][x[1]]
        tail_id = dicts['ent_id_1010'][x[2]]
        date_split = x[3].split('T')[0].split('-')
        date_id = dicts['ts_id_1010'][(int(date_split[0]), int(date_split[1]), int(date_split[2]))]

        try:
            end_date_split = x[4].split('T')[0].split('-')
            end_date_id = dicts['ts_id_1010'][(int(end_date_split[0]), int(end_date_split[1]), int(end_date_split[2]))]
        except:
            end_date_id = dicts['ts_id_1010'][(2021, 7, 1)]


        quad_id = (head_id, relation_id, tail_id, date_id, end_date_id)
        
        quad_ids.append(quad_id)
    
    return quad_ids

In [74]:
def prepare_datapoints(row):
    event_date = row['publishedAt']

    before_tkg = row['before_tkg']
    after_tkg = row['after_tkg']
    differences = row['differences']

    if type(row['latest_head_questions']) == float:
        before_head_questions = row['latest_head_questions']
        after_head_questions = row['latest_head_questions']
    else:
        before_head_questions = row['latest_head_questions']['before']
        after_head_questions = row['latest_head_questions']['after']

    if type(row['latest_tail_questions']) == float:
        before_tail_questions = row['latest_tail_questions']
        after_tail_questions = row['latest_tail_questions']
    else:
        before_tail_questions = row['latest_tail_questions']['before']
        after_tail_questions = row['latest_tail_questions']['after']

    return {
        'article_id': row['article_id'],
        'sentence_text': row['sentences'],
        'tuples': row['tuples'],
        'entities': ast.literal_eval(row['companies_deal']),
        'relations': row['relation'],
        'date': event_date,
#         'before_tkg': encoding_tkg(before_tkg),
#         'after_tkg': encoding_tkg(after_tkg),
        'before_tkg': before_tkg,
        'after_tkg': after_tkg,
        'differences_tkg': str(differences),
        'general_questions': row['latest_general_questions'],
        'before_head_questions': before_head_questions,
        'before_tail_questions': before_tail_questions,
        'after_head_questions': after_head_questions,
        'after_tail_questions': after_tail_questions,
           }

In [75]:
all_data = []
for index, row in sentence_df.iterrows():
    all_data.append(prepare_datapoints(row))

In [77]:
train = all_data[:4633]
val = all_data[4633:5148]
test = all_data[5148:]

In [54]:
from sklearn.model_selection import train_test_split

train_val_split, test_data  = train_test_split(sentence_df, train_size=0.9)
train_data, val_data = train_test_split(train_val_split, train_size=0.9)

In [55]:
len(train_data)

4633

In [56]:
len(val_data)

515

In [57]:
len(test_data)

573

In [58]:
train = []
for index, row in train_data.iterrows():
    train.append(prepare_datapoints(row))

val = []
for index, row in val_data.iterrows():
    val.append(prepare_datapoints(row))

test = []
for index, row in test_data.iterrows():
    test.append(prepare_datapoints(row))

In [84]:
for (dic, f) in zip([all_data, train, val, test, flatten_all_tkg_snapshots], ['tkg_all_data_1010.pickle', 'tkg_train_data_1010.pickle', 'tkg_val_data_1010.pickle', 'tkg_test_data_1010.pickle', 'all_tkg_snapshots_1010.pickle']):
    ff = open(os.path.join(data_path, f), 'wb')
    pickle.dump(dic, ff)
    ff.close()

In [85]:
data_dicts = {}
for f in ['tkg_all_data_1010.pickle', 'tkg_train_data_1010.pickle', 'tkg_val_data_1010.pickle', 'tkg_test_data_1010.pickle', 'all_tkg_snapshots_1010.pickle']:
    in_file = open(str(data_path + f), 'rb')
    data_dicts[f] = pickle.load(in_file)

In [86]:
print(len(data_dicts['tkg_all_data_1010.pickle']))
print(len(data_dicts['tkg_train_data_1010.pickle']))
print(len(data_dicts['tkg_val_data_1010.pickle']))
print(len(data_dicts['tkg_test_data_1010.pickle']))

5721
4633
515
573


In [ ]:
# for i in range(len(sentence_df)):
#     data = sentence_df.iloc[i]
#     head_entity = data['lowered_head']
#     relation_word, deal = data['relation'].split('_')[0], data['relation'].split('_')[1]
#     tail_entity = data['lowered_tail']
#     date = data['publishedAt'].split(" ")
#     date = date[0] + 'T' + date[1] + 'Z'
#     quad = (head_entity, data['relation'], tail_entity, date)
    
#     latest_dict = copy.deepcopy(all_tkg_snapshots[-1])
    
#     try:
#         head_data = latest_dict[head_entity]
#         tail_data = latest_dict[tail_entity]

#         common_entities = return_common_entities(head_data, tail_data)
        
#         if relation_word == 'success':
#             new_head_tuples = []
#             new_tail_tuples = []
#             to_be_remove_tuples = []

#             if deal == 'acq':
#                 # success_acq
#                 for d in tail_data:
#                     if d[1] == 'owner of (P1830)' or d[1] == 'subsidiary (P355)' or d[1] == 'business division (P199)':
#                         # the head_entity now acquires certain assets of the tail and new quads are being added
#                         new_tuple = (head_entity, d[1], d[2], date)
#                         new_head_tuples.append(new_tuple)

#                         # the tail entity is being acquired so all tuples now include an end date
#                         end_tuple = d + (date,)
#                         new_tail_tuples.append(end_tuple)
#                         to_be_remove_tuples.append(d)
#                     elif d[1] == 'board member (P3320)' or d[1] == 'owned by (P127)':
#                         if not d[2] in common_entities:
#                             # the tail entity is being acquired so all tuples now include an end date
#                             end_tuple = d + (date,)
#                             new_tail_tuples.append(end_tuple)
#                             to_be_remove_tuples.append(d)
                
#                 for h in head_data:
#                     # the tail entity is being acquired so all the board member and owned by values now own the tail entity
#                     if h[1] == 'board member (P3320)' or h[1] == 'owned by (P127)':
#                         new_tuple = (tail_entity, h[1], h[2], date)
#                         new_tail_tuples.append(new_tuple)

#                 # remove outdated tuples
#                 for z in to_be_remove_tuples:
#                     latest_dict[tail_entity].remove(z)

#                 # add new tuples
#                 latest_dict[head_entity].append(quad)
#                 latest_dict[head_entity] = latest_dict[head_entity] + new_head_tuples
#                 latest_dict[tail_entity] = latest_dict[tail_entity] + new_tail_tuples
#             elif deal == 'merger':
#                 # success_merger
#                 latest_dict[head_entity].append(quad)
#         else:
#             # all other relations
#             latest_dict[head_entity].append(quad)
#     except:
#         if head_entity in latest_dict.keys():
#             latest_dict[head_entity].append(quad)
#         else:
#             latest_dict[head_entity] = [quad]
        
#         if tail_entity not in latest_dict.keys():
#             latest_dict[head_entity] = []
    
#     all_tkg_snapshots.append(latest_dict)